In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img 
import matplotlib.cm as cm
import cv2
from scipy import ndimage as ndi

import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
import json

import os

from prettyprinter import pprint

In [2]:
def read_json(json_path, frame_num):
    f = open(json_path)
    dic = json.load(f)
    # frame_num to png_num in json hat 1 digi difference
    frame_num = frame_num -1
    
    # d return the dictionary for specific frame of a video
    d = dic[frame_num]
    anno = d.get('annotations')
    # anno contains multiple dictionaries of tool character points for ONE frame
    if len(anno) > 0:
        return anno
    else:
        return False
    
def write_list(tool_list, tool_class, tool_id, x, y):
    if tool_class == "LeftClasperPoint":
        if tool_id == "tool1":
            tool_list[0, 0] = x
            tool_list[1, 0] = y
        else:
            tool_list[0, 1] = x
            tool_list[1, 1] = y
    elif tool_class == "RightClasperPoint":
        if tool_id == "tool1":
            tool_list[0, 2] = x
            tool_list[1, 2] = y
        else:
            tool_list[0, 3] = x
            tool_list[1, 3] = y
    elif tool_class == "HeadPoint":
        if tool_id == "tool1":
            tool_list[0, 4] = x
            tool_list[1, 4] = y
        else:
            tool_list[0, 5] = x
            tool_list[1, 5] = y
    elif tool_class == 'ShaftPoint':
        if tool_id == "tool1":
            tool_list[0, 6] = x
            tool_list[1, 6] = y
        else:
            tool_list[0, 7] = x
            tool_list[1, 7] = y

    elif tool_class == "EndPoint":
        if tool_id == "tool1":
            tool_list[0, 8] = x
            tool_list[1, 8] = y
        else:
            tool_list[0, 9] = x
            tool_list[1, 9] = y
    return tool_list

import matplotlib.cm as cm
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float
from prettyprinter import pprint

def plt_multi(img,m,bin_den,max_draw):
    fig=plt.figure(figsize=(50, 15))
    columns = 5
    rows = 2
#     if mode:
#         print(m[0].shape)
#         m = tf.transpose(m, [0, 3, 1, 2])
    for i in range(0, rows*columns-1):
        fig.add_subplot(rows, columns, i+1).set_title(i)
        if max_draw:
            coordinates = peak_local_max(m[bin_den][0, :, :, i], min_distance=50, threshold_abs = 0.6)
            pprint(coordinates)
            pprint(coordinates[:, 1].shape[0])
            plt.scatter(coordinates[:, 1], coordinates[:, 0],s=10, c='m')
        plt.imshow(m[bin_den][0, :, :, i],cmap=cm.binary)#, alpha=0.2)
#         plt.imshow(img,cmap=cm.binary)
        
    fig.add_subplot(rows, columns, 10).set_title("img")
    plt.imshow(img,cmap=cm.binary)
    plt.show()
    
def plt_trt(m,img):
    fig=plt.figure(figsize=(50, 15))
    columns = 5
    rows = 2
#     m_sum = np.zeros((256,320))
    for i in range(0, columns*rows-1):
        fig.add_subplot(rows, columns, i+1).set_title(i)
        #plt.imshow(pre)
        plt.imshow(m[0][:, :, i],cmap=cm.binary)#, alpha=0.2)
#         m_sum = m_sum + m[0][:, :, i]

    fig.add_subplot(rows, columns, 10).set_title(10)
    plt.imshow(img,cmap=cm.binary)
    plt.show()

In [3]:
label_test_dir = '/tf/mount/Dataset/unused/robotic_bk/label/test'
video_test_dir = '/tf/mount/Dataset/unused/robotic_bk/video/test'

In [4]:
label_list = os.listdir(label_test_dir)
video_list = os.listdir(video_test_dir)

In [5]:
model = tf.keras.models.load_model('/tf/project/instrument-pose-tracking/src/tf_data/checkpoint/aug_proba_0.5_decay_default_model_rd34_sigma10_skip.ckpt', compile=False)

In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.keras.preprocessing import image
model_dir = '/tf/project/instrument-tracking/src/tf_data/checkpoint/aug_proba_0.5_decay_default_model_rd34_sigma10_skip.ckpt'

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode="FP16")
converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=model_dir,
    conversion_params=conversion_params)

converter.convert()
converter.save(output_saved_model_dir='./tf-converter')
from tensorflow.python.saved_model import tag_constants
model =  tf.saved_model.load('./tf-converter', tags =[tag_constants.SERVING])
signature_keys = list(model.signatures.keys())
# print(signature_keys)

infer = model.signatures["serving_default"]

INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)
INFO:tensorflow:Could not find TRTEngineOp_7 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_1 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_2 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_3 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_4 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Could not find TRTEngineOp_5 in 

In [6]:
def cal_score(sum_TP,sum_TN,sum_FP,sum_FN, sum_dis
              ,gt_mat, nn_mat):
    nn_mask = np.ones((2,10))*(nn_mat > 0)
    gt_mask = np.ones((2,10))*(gt_mat > 0)   
    
    distance = nn_mat*gt_mask - gt_mat*nn_mask
    distance = (distance[0]**2 + distance[1]**2)**(1/2) 
    
    TP_mask = gt_mask * nn_mask
    TN_mask = (np.ones((1,10))*(gt_mask * nn_mask == 0))-(np.ones((1,10))*(gt_mask - nn_mask !=0))
    
    FP_mask = np.ones((1,10))*distance > 30
    FP_mask = FP_mask +  np.ones((1,10))*(gt_mask-nn_mask>0)
    
    FN_mask = np.ones((1,10))*(gt_mask-nn_mask<0)
    
    

#     print(TP_mask,"\n",TN_mask,"\n",FP_mask,"\n",FN_mask)
    
    sum_dis += distance * (distance < 30)
        
    sum_TP += TP_mask
    sum_TN += TN_mask
    sum_FP += FP_mask
    sum_FN += FN_mask


#     print('nn_mat')
#     print(nn_mat.round(2))
    
#     print('gt_mat')
#     print(gt_mat.round(2))

#     print("==========||============")
#     print('distance')
#     print((distance * (distance < 30)).round(2) )
#     print("==========END============")
    
    return sum_TP, sum_TN, sum_FP, sum_FN, sum_dis

In [7]:
for t in range(0,5):
    print("test ", t+1)
    for i,vid in enumerate(video_list):
        vid_path = os.path.join(video_test_dir, vid)
        json_path = os.path.join(label_test_dir, vid.split('.')[0] + ".json")
        cap = cv2.VideoCapture(vid_path)
        print(vid)
        p=0
        sum_TP = np.zeros((2,10))
        sum_TN = np.zeros((2,10))
        sum_FP = np.zeros((2,10))
        sum_FN = np.zeros((2,10))
        sum_dis = np.zeros((1,10))
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                w = int(cap.get(3))
                h = int(cap.get(4))
                sca_w = w/320
                sca_h = h/256


                # print(frame_number)
                anno = read_json(json_path, frame_number)
                if anno:
                    nn_mat = np.zeros((2, 10))
                    gt_mat = np.zeros((2, 10))

                    frame= cv2.resize(frame,(320, 256))

    #                 img_nhwc = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.float32)
    #                 img_nchw = image.img_to_array(img_nhwc,data_format='channels_last')
    #                 img_nchw = img_nchw.reshape((1,256,320,3))
    #                 img_nchw = tf.constant(img_nchw)
    #                 m = infer(img_nchw)
    #                 m=m['y_dec_out'].numpy()

                    img_nhwc = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    img_nhwc = np.array([img_nhwc])
                    m = model.predict(img_nhwc)


                    #plt_multi(frame,m,1,1)
                    for layers in range(0, 5):
                        ly = m[0][0, :, :, layers]
    #                     ly = m[0][:, :, layers]


    #                     plt_trt(m,frame)
                        if layers != 4:
                            coordinates = peak_local_max(ly, min_distance=15, threshold_abs = 0.5,num_peaks =2)
                        elif layers == 4:
                            coordinates = peak_local_max(ly, min_distance=15, threshold_abs = 0.3,num_peaks = 2)

                        coordinates = np.rot90(coordinates)
    #                     coordinates= coordinates[: ,coordinates[0].argsort()]
                        coordinates = coordinates * [[sca_w],[sca_h]]

                        if coordinates.shape[1] == 2:
                            nn_mat[0:2, 2*layers:2*layers+2] = coordinates


    #                         plt.imshow(cv2.resize(ly,(w,h)),cmap=cm.binary)
    #                         plt.scatter(coordinates[0, :], coordinates[1, :],s=10)
    #                         plt.show()
                        elif coordinates.shape[1] == 1:
    #                         print(np.array(coordinates))
                            nn_mat[:, 2*layers] = np.transpose(coordinates)
                            nn_mat[:, 2*layers+1] = np.zeros((1,2))
    #                         plt.imshow(cv2.resize(ly,(w,h)),cmap=cm.binary)
    #                         plt.scatter(coordinates[0, :], coordinates[1, :],s=10)
    #                         plt.show()
                            #break
                        else:
    #                         pprint(coordinates)
        #                     plt.imshow(ly,cmap=cm.binary)
        #                     plt.scatter(coordinates[0, :], coordinates[1, :],s=10)
        #                     plt.show()
    #                         non_use = 1
                            break
                    for ele in anno:
                        tool_class = ele.get('class')
                        tool_id = ele.get('id')
                        x = int(ele.get('x'))
                        y = int(ele.get('y'))
                        gt_mat = write_list(gt_mat, tool_class, tool_id, x, y)


                    for column in range(0,11):
                        block = nn_mat[0:2, 2*column:2*column+2]
                        block= block[: ,block[0].argsort()]
                        nn_mat[0:2, 2*column:2*column+2] = block  

                    for column in range(0,11):
                        block = gt_mat[0:2, 2*column:2*column+2]
                        block= block[: ,block[0].argsort()]
                        gt_mat[0:2, 2*column:2*column+2] = block


                    sum_TP, sum_TN, sum_FP, sum_FN, sum_dis = cal_score(sum_TP,sum_TN,sum_FP,sum_FN, sum_dis,gt_mat, nn_mat)
                    p +=1

            else: 
                break
        cap.release()
        print("precision","\n", (sum_TP/(sum_TP+sum_FP)).round(2))
        print("recall","\n", (sum_TP/(sum_TP+sum_FN)).round(2))
        print("average pixel error","\n", (sum_dis/p).round(2))
        print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    

test  1
0_test.avi
precision 
 [[0.69 0.86 0.83 0.96 1.   0.99 0.78 0.92 0.71 0.64]
 [0.69 0.86 0.83 0.96 1.   0.99 0.78 0.92 0.71 0.64]]
recall 
 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
average pixel error 
 [[ 8.27 10.41  9.55 10.58  6.26  6.92  8.38  9.26 11.9   7.14]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
1_test.avi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in true_divide


precision 
 [[ nan 0.99  nan 1.    nan 0.99  nan 0.97  nan 0.08]
 [ nan 0.99  nan 1.    nan 0.99  nan 0.97  nan 0.08]]
recall 
 [[ 0.  1. nan  1.  0.  1.  0.  1. nan  1.]
 [ 0.  1. nan  1.  0.  1.  0.  1. nan  1.]]
average pixel error 
 [[ 0.   12.06  0.   10.    0.    9.84  0.   16.28  0.    1.66]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
2_test.avi
precision 
 [[ nan 0.99  nan 0.88  nan 0.86  nan 1.    nan 0.28]
 [ nan 0.99  nan 0.88  nan 0.86  nan 1.    nan 0.28]]
recall 
 [[ 0.  1.  0.  1.  0.  1.  0.  1. nan  1.]
 [ 0.  1.  0.  1.  0.  1.  0.  1. nan  1.]]
average pixel error 
 [[ 0.   13.4   0.    9.08  0.    8.36  0.   14.08  0.    4.59]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
3_test.avi
precision 
 [[ nan 1.    nan 0.96  nan 1.    nan 1.    nan 0.01]
 [ nan 1.    nan 0.96  nan 1.    nan 1.    nan 0.01]]
recall 
 [[ nan 1.    nan 0.97 0.   1.   0.   1.    nan 1.  ]
 [ nan 1.    nan 0.

precision 
 [[0.86 0.97 0.85 0.96 0.99 0.99 0.82 0.55 0.39 0.59]
 [0.86 0.97 0.85 0.96 0.99 0.99 0.82 0.55 0.39 0.59]]
recall 
 [[1.   1.   1.   1.   1.   1.   1.   1.   0.78 1.  ]
 [1.   1.   1.   1.   1.   1.   1.   1.   0.78 1.  ]]
average pixel error 
 [[ 9.99 11.31 11.24 10.24  8.89 11.29 17.22  2.65  3.75  7.16]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
5_test.avi
precision 
 [[0.93 0.97 0.91 0.96 0.97 0.98 0.79 0.55 0.12 0.42]
 [0.93 0.97 0.91 0.96 0.97 0.98 0.79 0.55 0.12 0.42]]
recall 
 [[1.   1.   1.   1.   1.   1.   1.   1.   0.76 1.  ]
 [1.   1.   1.   1.   1.   1.   1.   1.   0.76 1.  ]]
average pixel error 
 [[12.64  9.49 10.95 10.35 12.37 10.88 16.1   2.96  1.49  3.48]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
test  5
0_test.avi
precision 
 [[0.69 0.86 0.83 0.96 1.   0.99 0.78 0.92 0.71 0.64]
 [0.69 0.86 0.83 0.96 1.   0.99 0.78 0.92 0.71 0.64]]
recall 
 [[1. 1. 1. 1. 1. 1. 1. 1

In [75]:
a = np.random.randint(0,10,size=(2,10))
print(a)
a = a[: ,a[0].argsort()]
# np.count_nonzero(a)
print(a[:,2])

[[6 1 7 6 3 4 5 6 4 9]
 [7 2 3 0 8 6 4 3 8 5]]
[4 6]


array([0., 0.])

In [9]:
from sklearn.cluster import KMeans

for t in range(0,5):
    print("test ", t+1)
    non_use = 0
    for i,vid in enumerate(video_list):
        p=0
        sum_TP = np.zeros((2,10))
        sum_TN = np.zeros((2,10))
        sum_FP = np.zeros((2,10))
        sum_FN = np.zeros((2,10))
        sum_dis = np.zeros((1,10))
        vid_path = os.path.join(video_test_dir, vid)
        json_path = os.path.join(label_test_dir, vid.split('.')[0] + ".json")
    # vid_path = '/tf/mount/Dataset/unused/robotic_bk/video/test'+'/1_test.avi'
    # json_path = '/tf/mount/Dataset/unused/robotic_bk/label/test'+'/1_test.json'
        dis = np.zeros((1,10))
        count = 0
        cap = cv2.VideoCapture(vid_path)
        print(vid)
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                w = int(cap.get(3))
                h = int(cap.get(4))
                sca_w = w/320
                sca_h = h/256
                #frame = cv2.imread('./test_json_data/JPEGImages/000015.png')

                # print(frame_number)
                anno = read_json(json_path, frame_number)
                if anno:
                    nn_mat = np.zeros((2, 10))
                    gt_mat = np.zeros((2, 10))

                    frame= cv2.resize(frame,(320, 256))

#                     img_nhwc = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.float32)                
#                     img_nchw = image.img_to_array(img_nhwc,data_format='channels_last')
#                     img_nchw = img_nchw.reshape((1,256,320,3))
#                     img_nchw = tf.constant(img_nchw)
#                     m = infer(img_nchw)
#                     m=m['y'].numpy()

                    img_nhwc = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    img_nhwc = np.array([img_nhwc])
                    m = model.predict(img_nhwc)

                    #plt_multi(frame,m,1,1)
                    for layers in range(0, 5):
                        ly = m[0][0, :, :, layers]
#                         ly = m[0][:, :, layers]

                        _,ly = cv2.threshold(ly,0.6,1, cv2.THRESH_BINARY)
                        ly = np.transpose(np.nonzero(ly))
                        try:
                            kmeans = KMeans(n_clusters=2, random_state=0).fit(ly)
                            coordinates = kmeans.cluster_centers_  * [[sca_w],[sca_h]]
                            coordinates = np.rot90(coordinates)
        #                     coordinates = np.rot90(coordinates)
        #                     coordinates= coordinates[: ,coordinates[0].argsort()]
        #                     coordinates = coordinates * [[sca_w],[sca_h]]

                            if coordinates.shape[1] == 2:
                                nn_mat[0:2, 2*layers:2*layers+2] = coordinates

    #                             plt.imshow(cv2.resize(m[1][0, :, :, layers],(w,h)),cmap=cm.binary)
    #                             plt.scatter(coordinates[0, :], coordinates[1, :],s=10)
    #                             plt.show()
                            elif coordinates.shape[1] == 1:
        #                         print(coordinates.shape,layers)
                                nn_mat[:, 2*layers] = np.transpose(coordinates)
                                nn_mat[:, 2*layers+1] = np.zeros((1,2))
        #                         plt.imshow(ly,cmap=cm.binary)
        #                         plt.scatter(coordinates[0, :], coordinates[1, :],s=10)
        #                         plt.show()

                            else:
        #                         pprint(coordinates)
            #                     plt.imshow(ly,cmap=cm.binary)
            #                     plt.scatter(coordinates[0, :], coordinates[1, :],s=10)
            #                     plt.show()

                                break
                        except:
                            break

                    for ele in anno:
                        tool_class = ele.get('class')
                        tool_id = ele.get('id')
                        x = int(ele.get('x'))
                        y = int(ele.get('y'))
                        gt_mat = write_list(gt_mat, tool_class, tool_id, x, y)


                    for column in range(0,11):
                        block = nn_mat[0:2, 2*column:2*column+2]
                        block= block[: ,block[0].argsort()]
                        nn_mat[0:2, 2*column:2*column+2] = block    


                    for column in range(0,11):
                        block = gt_mat[0:2, 2*column:2*column+2]
                        block= block[: ,block[0].argsort()]
                        gt_mat[0:2, 2*column:2*column+2] = block


                    if gt_mat[0,0] == 0 and gt_mat[0,1] != 0:
                        for column in range(0,5):
                            block = nn_mat[0:2, 2*column:2*column+2]

                            block= np.array([[0, int(block[0,0]+block[0,1])/2],
                                             [0, int(block[1,0]+block[1,1])/2]])

                            nn_mat[0:2, 2*column:2*column+2] = block 



                    sum_TP, sum_TN, sum_FP, sum_FN, sum_dis = cal_score(sum_TP,sum_TN,sum_FP,sum_FN, sum_dis,gt_mat, nn_mat)
                    p +=1

            else: 
                break
        cap.release()
        print("precision","\n", (sum_TP/(sum_TP+sum_FP)).round(2))
        print("recall","\n",  (sum_TP/(sum_TP+sum_FN)).round(2))
        print("average pixel error","\n",  (sum_dis/p).round(2))
        print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
    #     print("==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")

test  1
0_test.avi
precision 
 [[1.   1.   0.99 0.98 1.   1.   1.   1.   1.   1.  ]
 [1.   1.   0.99 0.98 1.   1.   1.   1.   1.   1.  ]]
recall 
 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
average pixel error 
 [[5.27 6.24 5.83 5.49 2.45 2.7  2.11 2.69 3.79 8.28]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
1_test.avi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


precision 
 [[ nan 1.    nan 0.99  nan 1.    nan 1.    nan 1.  ]
 [ nan 1.    nan 0.99  nan 1.    nan 1.    nan 1.  ]]
recall 
 [[nan  1. nan  1. nan  1. nan  1. nan  1.]
 [nan  1. nan  1. nan  1. nan  1. nan  1.]]
average pixel error 
 [[0.   5.43 0.   6.76 0.   2.24 0.   3.27 0.   5.9 ]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
2_test.avi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


precision 
 [[ nan 0.97  nan 0.95  nan 1.    nan 1.    nan 1.  ]
 [ nan 0.97  nan 0.95  nan 1.    nan 1.    nan 1.  ]]
recall 
 [[nan  1. nan  1. nan  1. nan  1. nan  1.]
 [nan  1. nan  1. nan  1. nan  1. nan  1.]]
average pixel error 
 [[0.   4.55 0.   4.85 0.   2.26 0.   2.56 0.   9.53]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
3_test.avi


...

precision 
 [[ nan 1.    nan 0.99  nan 1.    nan 1.    nan 1.  ]
 [ nan 1.    nan 0.99  nan 1.    nan 1.    nan 1.  ]]
recall 
 [[nan  1. nan  1. nan  1. nan  1. nan  1.]
 [nan  1. nan  1. nan  1. nan  1. nan  1.]]
average pixel error 
 [[0.   5.43 0.   6.76 0.   2.24 0.   3.27 0.   5.9 ]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
2_test.avi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


precision 
 [[ nan 0.97  nan 0.95  nan 1.    nan 1.    nan 1.  ]
 [ nan 0.97  nan 0.95  nan 1.    nan 1.    nan 1.  ]]
recall 
 [[nan  1. nan  1. nan  1. nan  1. nan  1.]
 [nan  1. nan  1. nan  1. nan  1. nan  1.]]
average pixel error 
 [[0.   4.55 0.   4.85 0.   2.26 0.   2.56 0.   9.53]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
3_test.avi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide


precision 
 [[ nan 1.    nan 0.97  nan 0.9   nan 1.    nan 1.  ]
 [ nan 1.    nan 0.97  nan 0.9   nan 1.    nan 1.  ]]
recall 
 [[ nan 1.    nan 0.97  nan 1.    nan 1.    nan 1.  ]
 [ nan 1.    nan 0.97  nan 1.    nan 1.    nan 1.  ]]
average pixel error 
 [[0.   4.51 0.   6.09 0.   3.24 0.   2.72 0.   7.12]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
4_test.avi
precision 
 [[0.98 0.99 0.96 0.95 0.98 0.99 0.99 0.99 0.99 0.78]
 [0.98 0.99 0.96 0.95 0.98 0.99 0.99 0.99 0.99 0.78]]
recall 
 [[1.   1.   1.   1.   1.   1.   1.   1.   0.58 1.  ]
 [1.   1.   1.   1.   1.   1.   1.   1.   0.58 1.  ]]
average pixel error 
 [[6.34 5.44 9.29 7.1  3.38 2.67 3.74 2.8  2.42 6.62]]
==========!!!!!!!!!!!!!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@!!!!!!!!!!!!============
5_test.avi
precision 
 [[0.98 0.98 0.97 0.96 0.96 0.96 0.94 0.97 0.85 0.77]
 [0.98 0.98 0.97 0.96 0.96 0.96 0.94 0.97 0.85 0.77]]
recall 
 [[1.   1.   1.   1.   1.   1.   1.   1.   0.75 1.  ]

In [65]:
a = np.array([0,0,1,1,0,0])
b = np.array([0,0,1,1,0,1])
print(np.ones((1,6))*(a*b == 0)-(np.ones((1,6))*(a-b !=0)))

(np.ones((1,6))*(a>1)).shape

[[1. 1. 0. 0. 1. 0.]]


(1, 6)